https://aws.amazon.com/blogs/machine-learning/create-and-fine-tune-sentence-transformers-for-enhanced-classification-accuracy/

In [1]:
import numpy as np
import torch
import torchmetrics
from sentence_transformers import SentenceTransformer
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import re
from torch import nn, optim
import torchmetrics
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, Specificity, AUROC, ROC
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
# 1. Load a pretrained Sentence Transformer model
# model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
# embedding_size = 384


In [82]:
# print(f"Max seq len = {model.max_seq_length}")
# sentences = ["This is sentence 1.",
#              "This is sentence 2."]

# embeddings = model.encode(sentences)
# print(type(embeddings))
# print(embeddings.shape)

# similarities = model.similarity(embeddings, embeddings)
# print(similarities)

Max seq len = 256
<class 'numpy.ndarray'>
(2, 384)
tensor([[1.0000, 0.9287],
        [0.9287, 1.0000]])


In [83]:
model.prompts
max seq len and truncation

{}

In [11]:
num_classes = 9
# embedding_size = model.get_sentence_embedding_dimension()
embedding_size = 384

class ClassificationHead(nn.Module):
    def __init__(self, embedding_size: int, num_classes: int):
        super(ClassificationHead, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(embedding_size, num_classes),
        )
    
    def forward(self, features: torch.Tensor):
        x = features["sentence_embedding"]
        x = self.layers(x)
        return x

class SentenceTransformerWithHead(nn.Module):
    def __init__(self, transformer: SentenceTransformer, head: ClassificationHead):
        super(SentenceTransformerWithHead, self).__init__()
        self.transformer = transformer
        self.head = head
    
    def forward(self, input: dict[torch.Tensor, torch.Tensor]):
        # input['input_ids']
        # input['attention_mask']
        features = self.transformer(input)
        logits = self.head(features)
        return logits

# head = ClassificationHead(embedding_size, num_classes)
# model_with_head = SentenceTransformerWithHead(model, head)
# head.to(device)
# model_with_head.to(device)
        

In [85]:
# print(next(model.parameters()).device)
# print(next(head.parameters()).device)
# print(next(model_with_head.parameters()).device)

cuda:0
cuda:0
cuda:0


In [46]:
# inputs = model.tokenize(sentences)
# inputs = {key: value.to(device) for key, value in inputs.items()}
# inputs

{'input_ids': tensor([[ 101, 2023, 2003, 6251, 1015, 1012,  102],
         [ 101, 2023, 2003, 6251, 1016, 1012,  102]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [47]:
# logits = model_with_head(inputs)

In [19]:
class QueryDataset(Dataset):
    def __init__(self, csv_file: str | None=None, df: pd.core.frame.DataFrame | None=None):
        if csv_file is not None:
            self.df = pd.read_csv(csv_file)
        elif df is not None:
            self.df = df
        self.df = self.df.astype({"Label": "int64"})
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        item = {
            "Query": row["Query"],
            "Label": row["Label"],

        }
        return item

In [16]:
df = pd.read_csv("intent_classifier_samples.csv")

train_idx, val_idx = train_test_split(range(len(df)), test_size=0.3, stratify=df["Label"], random_state=0)
df_train = df.iloc[train_idx]
val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=df.iloc[val_idx]["Label"], random_state=0)
df_val = df.iloc[val_idx]
df_test = df.iloc[test_idx]
# print(df_train["Label"].value_counts())
# print(df_val["Label"].value_counts())
# print(df_test["Label"].value_counts())
# print(df_train.head())
# print(df_val.head())
# print(df_test.head())

In [20]:
ds_train = QueryDataset(df=df_train)
ds_val = QueryDataset(df=df_val)
ds_test = QueryDataset(df=df_test)

In [21]:
batch_size = 64
train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(ds_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False)
sample = next(iter(train_loader))
sample

{'Query': ['In the mood for a gripping thriller series with unexpected plot twists and morally ambiguous characters, any recommendations?',
  "I'm content with my literary picks for now, so I'll skip on any new recommendations.",
  "Hey there, I'm in the mood for a book that will challenge my beliefs. Can you recommend something that will provoke my thoughts?",
  'Can you recommend a book that evokes emotions like Where the Crawdads Sing by Delia Owens?',
  "Just finished The Night Circus and I'm spellbound! Any recommendations for a book with a similarly enchanting and magical atmosphere?",
  'Can you recommend historical fiction books spotlighting the lives of extraordinary women throughout different eras in history?',
  'How does your system handle user inquiries regarding the rationale behind specific book recommendations?',
  'Do you have books in other languages?',
  "Let's take a moment to revisit the books we've touched upon earlier, shall we?",
  'What percentage of books in t

In [22]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [ ]:
# sample["Query"]
# sample["Label"]
# model.tokenize(sample["Query"])

In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
head = ClassificationHead(embedding_size, num_classes)
model_with_head = SentenceTransformerWithHead(model, head)
head.to(device)
model_with_head.to(device)

SentenceTransformerWithHead(
  (transformer): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
    (2): Normalize()
  )
  (head): ClassificationHead(
    (layers): Sequential(
      (0): Linear(in_features=384, out_features=9, bias=True)
    )
  )
)

In [25]:
next(model_with_head.parameters()).device.type

'cuda'

In [15]:
model_with_head.transformer.tokenize

<bound method SentenceTransformer.tokenize of SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)>

In [132]:
# Freeze all weights except classifier head
for name, param in model_with_head.named_parameters():
    # print(name)
    if not re.search(r"^head", name):
        # print(name)
        param.requires_grad = False

for name, param in model_with_head.named_parameters():
    if param.requires_grad:
        print(name)

head.layers.0.weight
head.layers.0.bias


In [27]:
f1 = F1Score(task="multiclass", num_classes=num_classes, average="macro")
f1.to(device)
accuracy = Accuracy(task="multiclass", num_classes=num_classes, average="macro")
accuracy.to(device)
f1._get_name()
accuracy._get_name()

'MulticlassAccuracy'

In [133]:
loss_function = nn.CrossEntropyLoss(reduction="sum")
optimizer = optim.AdamW(model_with_head.parameters(), lr=0.01) # 0.01 for head only, 0.0001 for all weights
# lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.999)

f1 = F1Score(task="multiclass", num_classes=num_classes, average="macro")
f1.to(device)
accuracy = Accuracy(task="multiclass", num_classes=num_classes, average="macro")
accuracy.to(device)

metrics = [accuracy, f1]

n_epochs = 20
training_results = {
    "Train Loss": np.ones(n_epochs)*-999,
    "Train Accuracy": np.ones(n_epochs)*-999,
    "Train F1": np.ones(n_epochs)*-999,
    "Val Loss": np.ones(n_epochs)*-999,
    "Val Accuracy": np.ones(n_epochs)*-999,
    "Val F1": np.ones(n_epochs)*-999,
}
for edx in range(n_epochs):
    cum_loss = 0
    accuracy.reset()
    for bdx, batch in enumerate(train_loader):
        optimizer.zero_grad()

        inputs = model.tokenize(batch["Query"])
        inputs = {key: value.to(device) for key, value in inputs.items()}
        labels = batch["Label"].to(device)

        logits = model_with_head(inputs)

        loss = loss_function(logits, labels)
        cum_loss += loss.item()
        loss.backward()
        optimizer.step()
        # lr_scheduler.step()

        for metric in metrics:
            metric.update(torch.argmax(logits, dim=1), labels)
        # accuracy.update(torch.argmax(logits, dim=1), labels)
    
    t_loss = cum_loss/len(train_loader.dataset.df)
    t_acc = accuracy.compute().detach().cpu().numpy().item()
    t_f1 = f1.compute().detach().cpu().numpy().item()
    training_results["Train Loss"][edx] = t_loss
    training_results["Train Accuracy"][edx] = t_acc
    training_results["Train F1"][edx] = t_f1

    cum_loss = 0
    accuracy.reset()
    with torch.no_grad():
        for bdx, batch in enumerate(val_loader):
            inputs = model.tokenize(batch["Query"])
            inputs = {key: value.to(device) for key, value in inputs.items()}
            labels = batch["Label"].to(device)

            logits = model_with_head(inputs)

            loss = loss_function(logits, labels)
            cum_loss += loss.item()

            for metric in metrics:
                metric.update(torch.argmax(logits, dim=1), labels)
    
    v_loss = cum_loss/len(val_loader.dataset.df)
    v_acc = accuracy.compute().detach().cpu().numpy().item()
    v_f1 = f1.compute().detach().cpu().numpy().item()
    training_results["Val Loss"][edx] = v_loss
    training_results["Val Accuracy"][edx] = v_acc
    training_results["Val F1"][edx] = v_f1

    print(f"Train Loss: {t_loss:.2f}, Train Accuracy: {t_acc:.2f}, Train F1: {t_f1:.2f}, \
          Val Loss: {v_loss:.2f}, Val Accuracy: {v_acc:.2f}, Val F1: {v_f1:.2f}")

print("Training finished.")

Train Loss: 1.75, Train Accuracy: 0.40, Train F1: 0.43,           Val Loss: 1.36, Val Accuracy: 0.68, Val F1: 0.49
Train Loss: 1.10, Train Accuracy: 0.83, Train F1: 0.67,           Val Loss: 0.91, Val Accuracy: 0.84, Val F1: 0.69
Train Loss: 0.76, Train Accuracy: 0.86, Train F1: 0.75,           Val Loss: 0.69, Val Accuracy: 0.91, Val F1: 0.76
Train Loss: 0.57, Train Accuracy: 0.93, Train F1: 0.80,           Val Loss: 0.56, Val Accuracy: 0.92, Val F1: 0.81
Train Loss: 0.46, Train Accuracy: 0.94, Train F1: 0.83,           Val Loss: 0.47, Val Accuracy: 0.92, Val F1: 0.84
Train Loss: 0.39, Train Accuracy: 0.94, Train F1: 0.85,           Val Loss: 0.41, Val Accuracy: 0.93, Val F1: 0.86
Train Loss: 0.34, Train Accuracy: 0.95, Train F1: 0.87,           Val Loss: 0.37, Val Accuracy: 0.95, Val F1: 0.87
Train Loss: 0.30, Train Accuracy: 0.96, Train F1: 0.88,           Val Loss: 0.34, Val Accuracy: 0.95, Val F1: 0.88
Train Loss: 0.27, Train Accuracy: 0.96, Train F1: 0.89,           Val Loss: 0.31

In [34]:
labels

tensor([7, 7, 7, 7, 7, 0, 7, 7, 7, 5, 3, 6, 7, 6, 7, 7], device='cuda:0',
       dtype=torch.int32)

In [103]:
torch.argmax(logits, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')